In [1]:
#!pip install Census
#!pip install us 
#!pip install plotly==4.1.0

# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import us
from census import Census
from us import states
#from census_data import 

# Census API Key
from config import api_key
api_key
c = Census(api_key, year=2013)
c1 = Census(api_key,year=2017)

In [10]:
male_data=c1.acs1.get('B01003_001E', {'for': 'state:*'})
m_pd = pd.DataFrame(male_data)
m_pd = m_pd.rename(columns={"B01003_001E": "Population", 
                                       "state": "State Code"})
m_pd = m_pd[["State Code","Population"]]
m_pd=m_pd.fillna(0)
m_pd.head()
m_pd.to_csv("US_Population.csv", encoding="utf-8", index=False)

In [11]:
female_data=c1.acs1.get('B24010_155E', {'for': 'state:*'})
f_pd = pd.DataFrame(female_data)
f_pd = f_pd.rename(columns={"B24010_155E": "Female", 
                                       "state": "State Code"})
f_pd = f_pd[["State Code","Female"]]
f_pd.head()

,State Code,Female
0,01,123251.0
1,02,21991.0
2,04,221393.0
3,05,72013.0
4,06,1348070.0


In [12]:
us.states.mapping('fips', 'abbr')['01']

'AL'

In [13]:
def func_merge_df(df1, df2, column):
    mf_df = pd.merge(df1, df2, on=column, how='inner')
    return mf_df

def func_getdata(key1, key2, jobtype):
    male_data= []
    female_data = []
    state_dict = us.states.mapping('fips', 'name')
    
    male_data=c1.acs1.get(key1, {'for': 'state:*'})
    female_data=c1.acs1.get(key2, {'for': 'state:*'})
    m_pd = pd.DataFrame(male_data)
    f_pd = pd.DataFrame(female_data)
    # Merge the dataframe
    mf_df = func_merge_df(df1=f_pd, df2=m_pd, column='state')
    mf_df=mf_df.fillna(0)
    # Get the total 
    mf_df["State Name"] = ''
    mf_df["Type"] = ''
    # Get the State name
    for index, row in mf_df.iterrows():
        mf_df.loc[index,'State Name'] = state_dict[mf_df.loc[index,'state']]
        mf_df.loc[index,'Type'] = jobtype
    return mf_df

   
#rs_data = func_getdata(key1 = 'B24040_007E', key2 = 'B24040_111E', jobtype = 'Manufacturing')
#rs_data = rs_data.rename(columns={"B24040_007E": "Male","B24040_111E":"Female","state": "State Code"})
#rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
#rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
#rs_data.sort_values('Total', ascending=False).head() 
#rs_data

In [7]:
cnt = 0
input_data=[{'Male':'B24040_007E','Female':'B24040_111E','jobtype':'Manufacturing'},
           {'Male':'B24010_019E','Female':'B24010_170E','jobtype':'IT'},
           {'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}
          ]
input_df = pd.DataFrame(input_data)

for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    rs_data.sort_values('Total', ascending=False).head() 
    if cnt !=0:
        cnt = cnt+1

# First time for the header
    if cnt == 0:
        rs_data.to_csv("job_census_data_3.csv", encoding="utf-8", index=False)
        cnt = cnt + 1 

    if cnt > 1:
        with open('job_census_data_3.csv', 'a') as f:
            rs_data.to_csv(f, header=False, index=False)

print("Job Completed")

Job Completed


In [14]:
#top states.
cnt = 0

input_data=[{'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'}]
#input_data=[{'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}]
input_df = pd.DataFrame(input_data)

for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    final_df = rs_data.sort_values('Total', ascending=False).head() 
    
final_df

,State Code,State Name,Type,Male,Female,Total
4,06,California,Social Workers,66326.0,185081.0,251407
32,36,New York,Social Workers,45515.0,137849.0,183364
43,48,Texas,Social Workers,28644.0,105919.0,134563
38,42,Pennsylvania,Social Workers,29250.0,74910.0,104160
9,12,Florida,Social Workers,23239.0,76424.0,99663


In [4]:
# Social Workers
cnt = 1
input_data=[{'Male':'B24010_066E','Female':'B24010_217E','jobtype':'Psychiatric'},
           {'Male':'B24010_038E','Female':'B24010_189E','jobtype':'Religious'},
           {'Male':'B24010_037EA','Female':'B24010_188E','jobtype':'Counsolers'},
           {'Male':'B24010_091E','Female':'B24010_242E','jobtype':'Child Care'}
          ]
input_df = pd.DataFrame(input_data)
input_df
for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    rs_data.sort_values('Total', ascending=False).head() 

# First time for the header
    if cnt == 1:
        rs_data.to_csv("job_SocialWorker_data.csv", encoding="utf-8", index=False)
        cnt = cnt + 1

    if cnt > 1:
        with open('job_SocialWorker_data.csv', 'a') as f:
            rs_data.to_csv(f, header=False, index=False)

print("Job Completed")

Job Completed


In [5]:
# Social Workers
cnt = 1
input_data=[{'Male':'B24010_066E','Female':'B24010_217E','jobtype':'Psychiatric'},
           {'Male':'B24010_038E','Female':'B24010_189E','jobtype':'Religious'},
           {'Male':'B24010_037EA','Female':'B24010_188E','jobtype':'Counsolers'},
           {'Male':'B24010_091E','Female':'B24010_242E','jobtype':'Child Care'}
          ]
input_df = pd.DataFrame(input_data)
input_df
for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    
rs_data.sort_values('Total', ascending=False).head() 

,State Code,State Name,Type,Male,Female,Total
4,06,California,Child Care,14948.0,153040.0,167988
32,36,New York,Child Care,6629.0,107009.0,113638
43,48,Texas,Child Care,7149.0,98652.0,105801
13,17,Illinois,Child Care,2778.0,54157.0,56935
9,12,Florida,Child Care,3887.0,49141.0,53028


In [9]:
female_data=c1.acs1.get('B24010_189E', {'for': 'state:*'})
f_pd = pd.DataFrame(female_data)
f_pd = f_pd.rename(columns={"B24010_189E": "Female", 
                                       "state": "State Code"})
f_pd = f_pd[["State Code","Female"]]
f_pd.head()

,State Code,Female
0,01,3609.0
1,02,551.0
2,04,3209.0
3,05,1972.0
4,06,20834.0


In [11]:
# Finance
cnt = 1
input_data=[{'Male':'B24010_105E','Female':'B24010_256E','jobtype':'Audit Clerk'},
           {'Male':'B24010_016E','Female':'B24010_167E','jobtype':'Specialist'},
           {'Male':'B24010_008E','Female':'B24010_159E','jobtype':'Finance Manager'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Accountants'}
          ]
input_df = pd.DataFrame(input_data)
input_df
for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    rs_data.sort_values('Total', ascending=False).head() 

# First time for the header
    if cnt == 1:
        rs_data.to_csv("job_Finance_data.csv", encoding="utf-8", index=False)
        cnt = cnt + 1

    if cnt > 1:
        with open('job_Finance_data.csv', 'a') as f:
            rs_data.to_csv(f, header=False, index=False)

print("Job Completed")

Job Completed


In [4]:
# Finance
cnt = 1
input_data=[{'Male':'B24010_105E','Female':'B24010_256E','jobtype':'Audit Clerk'},
           {'Male':'B24010_016E','Female':'B24010_167E','jobtype':'Specialist'},
           {'Male':'B24010_008E','Female':'B24010_159E','jobtype':'Finance Manager'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Accountants'}
          ]
 
key1_val = 'B24010_105E'
key2_val = 'B24010_256E'
jobtypeVal = 'Audit Clerk'

rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
rs_data
rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
#rs_data.sort_values('Total', ascending=False).head() 
filter_df = rs_data[rs_data['State Name']=='California']
filter_df

,State Code,State Name,Type,Male,Female,Total
4,06,California,Audit Clerk,26429.0,109060.0,135489


In [7]:
#top states
cnt = 0
input_data=[{'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'}]
#input_data=[{'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}]
input_df = pd.DataFrame(input_data)
for index, row in input_df.iterrows():
   filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
   key1_val = ','.join(str(v) for v in filter_df['Male'])
   key2_val = ','.join(str(v) for v in filter_df['Female'])
   jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
   rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
   rs_data
   rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
   rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
   rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
   final_df = rs_data.sort_values('Total', ascending=False).head()
final_df

,State Code,State Name,Type,Male,Female,Total
4,06,California,Social Workers,66326.0,185081.0,251407
32,36,New York,Social Workers,45515.0,137849.0,183364
43,48,Texas,Social Workers,28644.0,105919.0,134563
38,42,Pennsylvania,Social Workers,29250.0,74910.0,104160
9,12,Florida,Social Workers,23239.0,76424.0,99663


In [14]:
#get the top 5 across all the category
cnt = 0
input_data=[{'Male':'B24040_007E','Female':'B24040_111E','jobtype':'Manufacturing'},
           {'Male':'B24010_019E','Female':'B24010_155E','jobtype':'IT'},
           {'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}
          ]
input_df = pd.DataFrame(input_data)

for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    if cnt == 0:
        final_data = rs_data.sort_values('Total', ascending=False).head()
        all_df = rs_data
        cnt += 1
    else:
        final_data = final_data.append(rs_data.sort_values('Total', ascending=False).head())
        all_df = all_df.append(rs_data)
final_data.head()

,State Code,State Name,Type,Male,Female,Total
4,06,California,Manufacturing,1020705.0,421201.0,1441906
43,48,Texas,Manufacturing,730346.0,242316.0,972662
22,26,Michigan,Manufacturing,570745.0,191694.0,762439
35,39,Ohio,Manufacturing,543443.0,187144.0,730587
13,17,Illinois,Manufacturing,463010.0,187851.0,650861


In [48]:
#us.states.mapping('fips', 'abbr')['01']

#','.join(str(v) for v in final_data['State Code'])
us.states.MD.fips
us.states.mapping('fips', 'abbr')[us.states.MD.fips]
abbr_dict=us.states.mapping('abbr', 'name')
abbr_dict

{'AL': 'Alabama',
 'AK': 'Alaska',
 'AS': 'American Samoa',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DK': 'Dakota',
 'DE': 'Delaware',
 'DC': 'District of Columbia',
 'FL': 'Florida',
 'GA': 'Georgia',
 'GU': 'Guam',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'MP': 'Northern Mariana Islands',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'OL': 'Orleans',
 'PA': 'Pennsylvania',
 'PI': 'Philippine Islands',
 'PR': 'Puerto Rico',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 

In [65]:
# get n largest jobs across   
#n_df=final_data.nlargest(5, 'Total')
#n_df
#all_df[['State Name','Type','Total']]
#all_df.head()
#all_df.loc[all_df['State Name'] == 'California']

df = pd.DataFrame([])
states_cd = all_df['State Name'].unique()

for x in range(len(states_cd)):
    #df = df.append({'State Name': states_cd[x]},ignore_index=True)
    df.loc[x,'State Name'] = states_cd[x]
    # get the two letter abbrevation from the state dictionary 
    for key,val in abbr_dict.items():
        if states_cd[x] == val:
            df.loc[x,'State Code'] = key
    df.loc[x,'Manufacturing'] = 0
    df.loc[x,'IT'] = 0
    df.loc[x,'Finance'] = 0
    df.loc[x,'Social Workers'] = 0
    df.loc[x,'Total Jobs'] = 0
    
for index, row in df.iterrows():
    # df.set_value(i,'ifor',ifor_val) using set_value re-arrange the data
    df1=all_df.loc[all_df['State Name'] == df.loc[index,'State Name']]
    #df.set_value(index,'State Code',us.states.mapping('fips', 'abbr')[df1.loc[df1['Type'] == 'Manufacturing']['Total']])
    df.set_value(index,'Manufacturing',df1.loc[df1['Type'] == 'Manufacturing']['Total'])
    df.set_value(index,'IT',df1.loc[df1['Type'] == 'IT']['Total'])
    df.set_value(index,'Finance',df1.loc[df1['Type'] == 'Finance']['Total'])
    df.set_value(index,'Social Workers',df1.loc[df1['Type'] == 'Social Workers']['Total'])
    total_jobs = df1.loc[df1['Type'] == 'Manufacturing']['Total'] + df1.loc[df1['Type'] == 'IT']['Total'] + df1.loc[df1['Type'] == 'Social Workers']['Total'] + df1.loc[df1['Type'] == 'Finance']['Total']
    df.set_value(index,'Total Jobs',total_jobs)

df.head()

/Users/Shiva/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

/Users/Shiva/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

/Users/Shiva/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

/Users/Shiva/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

/Users/Shiva/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning:

set_value i

,State Name,State Code,Manufacturing,IT,Finance,Social Workers,Total Jobs
0,Alabama,AL,257225.0,195887.0,21917.0,23442.0,498471.0
1,Alaska,AK,0.0,35739.0,4588.0,7927.0,48254.0
2,Arizona,AZ,182707.0,351589.0,39870.0,38458.0,612624.0
3,Arkansas,AR,152729.0,105955.0,11413.0,15991.0,286088.0
4,California,CA,1441906.0,2270443.0,262401.0,251407.0,4226157.0


In [63]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd
init_notebook_mode(connected=True)


for col in df.columns:
    df[col] = df[col].astype(str)
    
# assign color 
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]


df['text'] = df['State Name'] + '<br>' +\
    'Manufacturing '+df['Manufacturing']+'<br>'+\
    'IT '+df['IT']+'<br>'+\
    'Finance '+df['Finance'] +'<br>'+\
    'Social Workers ' + df['Social Workers']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['State Code'],
        z = df['Total Jobs'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Million Jobs"
        )
    ) ]

layout = dict(
        title = 'US Job Trend Distributed Across all States (Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )

fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )